In [1]:
from pymongo import MongoClient
import json

mongoClient = MongoClient('192.168.194.171', port=27017)
db = mongoClient.db_restT



In [7]:
users = db.retweets.find( {}, { "userid" : 1 } )

In [ ]:
users = db.restT.distinct( "user.id_str", { "lang": "en" } )

In [3]:
print type(users)

<class 'pymongo.cursor.Cursor'>


In [8]:
for user in users:
    print user['userid']

26270913
26270913
26270913
26270913
26270913
26270913
2238576858
26270913
487393551
26270913
8521542
26270913
26270913
26270913
840957998
26270913
31451582
313162621
487393551
330446733
313005140
26270913
68577152
1049422088
353060311
805230144
831981271
2990999770
26270913
1321629762


In [4]:
import tweepy
import os
import re

consumer_key = os.getenv('TWITTER_APP_KEY')
consumer_secret = os.getenv('TWITTER_CONSUMER_SECRET')

auth = tweepy.AppAuthHandler(consumer_key, consumer_secret)
api = tweepy.API(auth_handler=auth,wait_on_rate_limit=True,wait_on_rate_limit_notify=True)

_text = None

try:
    for status in tweepy.Cursor(api.user_timeline, id=2872556463, count=3500).items(3500):
        t = re.sub(r'https?:\/\/(\S*)', '', status.text)
        if not _text:
            _text = t
        else:
            _text += t
except tweepy.TweepError as te:
    if te.reason.lower().find("not authorized"):
        print "user not authorized, skipping..."



/Users/rcordell/Documents/MIDS/W205/W205/lib/python2.7/site-packages/requests/packages/urllib3/util/ssl_.py:90: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning
/Users/rcordell/Documents/MIDS/W205/W205/lib/python2.7/site-packages/requests/packages/urllib3/util/ssl_.py:90: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning


user not authorized, skipping...


In [16]:
print _text

RT @JoeStrauss: Cards blow lead. Focus immediately shifts to umpire. Water is wet.Pray for your pastor. Chances are he or she is very unhealthy, in part from the demands of ministry: You are not alone.RT @timkellernyc: We see the Bible as a book of answers to our questions, but if we let the text speak we find that we are not even asking …had fun with all the #IGRC2015 tweeps - until next year...Our bishop, to Australian @mikerayson "Yes, you waving the green card." True on so many levels. #IGRC2015"Hey you...over there...can you pray?" @ILAreaBishop to @jeremylafary #IGRC2015@AlexisJomae never forget that old men were once young kids too (and probably went to camp)! ;-) #IGRC2015@RevNicoleCox you should meet my wife! ;-) Cuteness overload!!!Married clergy and clergy parents - lets not forget our "ministry" is primarily under our own roof. #IGRC2015 RT @danigirl139: "Please find the kids now! Bring them in now! Right now!" --Bishop Keaton #IGRC2015@RevNicoleCox only when you're as awes

In [30]:
from nltk import word_tokenize
from nltk import FreqDist
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import re
from collections import Counter
corpus = []
stop = stopwords.words('english')
stop.extend(['rt','http','https','co','1','2','3','4','5','6','7','8','9'])
tokenizer = RegexpTokenizer(r'\w+')

corpus.extend([i for i in tokenizer.tokenize(_text.lower()) if i not in stop])
c = Counter(corpus)

lexicon = {}
lexicon['user_id'] = users[0]
lexicon['word_count'] = len(corpus)
lexicon['postings'] = dict(c)
lexicon['unique_words'] = len(dict(c))
lexicon['diversity'] = float(lexicon['unique_words'])/float(lexicon['word_count'])

In [31]:
lexicon

{'diversity': 0.312815008400224,
 'postings': {u'notree': 1,
  u'yellow': 2,
  u'four': 10,
  u'igrc2015repentance': 1,
  u'woods': 2,
  u'jesusif': 1,
  u'schnucks': 1,
  u'captain': 1,
  u'dalercohen': 1,
  u'gc2012independent': 1,
  u'jtshaver': 2,
  u'buddy': 2,
  u'eligible': 2,
  u'electricity': 1,
  u'rouse': 1,
  u'whoso': 1,
  u'nflonwciart': 1,
  u'presents': 2,
  u'iancron': 1,
  u'lord': 19,
  u'igrc2013thanks': 1,
  u'recoveryshake': 1,
  u'shaving': 1,
  u'sinking': 2,
  u'rescue': 1,
  u'cavaliers': 1,
  u'zackaryglenn': 2,
  u'uofi': 1,
  u'smack': 2,
  u'foul': 1,
  u'affect': 1,
  u'bringing': 6,
  u'courageous': 7,
  u'matzah': 2,
  u'jrrtolkien': 8,
  u'prize': 1,
  u'holiness': 4,
  u'skills': 1,
  u'thricemantis': 1,
  u'companies': 1,
  u'wednesday': 1,
  u'wcia_dfabert': 2,
  u'indefinitely': 1,
  u'shouting': 1,
  u'wallswhen': 1,
  u'clothes': 1,
  u'shockernationtime': 1,
  u'igrc2011dominant': 1,
  u'force': 1,
  u'ammonium': 1,
  u'tired': 4,
  u'miller': 3

In [29]:
print float(lexicon['unique_words'])/float(lexicon['word_count'])

0.3128150084


In [13]:
docs = db.lexdiv2.aggregate( 
    [
        { "$group" : { "_id" : "$user_id",
                       "diversity" : {"$first" : "$diversity"},
                       "followers_count" : { "$first" : "$followers_count"} }},
        { "$sort" :  { "followers_count" : -1}},
        { "$project" : { "diversity" : 1,
                        "followers_count" : 1,
                         "_id" : 1}}
    ])

In [14]:
max_followers = 0
lex = {}
follower_lex = []
for doc in docs:
    print doc
    lex[doc['_id']] = doc['diversity']
    follower_lex.append((doc['followers_count'], doc['diversity']))
    if doc['followers_count'] > max_followers:
        max_followers = doc['followers_count']

{u'followers_count': 3065367, u'_id': u'330446733', u'diversity': 0.31103000811030007}
{u'followers_count': 2049528, u'_id': u'805230144', u'diversity': 0.1545284664258129}
{u'followers_count': 1749003, u'_id': u'31451582', u'diversity': 0.23825352618812326}
{u'followers_count': 1012321, u'_id': u'26270913', u'diversity': 0.22243024963289282}
{u'followers_count': 927141, u'_id': u'2238576858', u'diversity': 0.4324224519940916}
{u'followers_count': 653605, u'_id': u'256495314', u'diversity': 0.33512624714341455}
{u'followers_count': 574639, u'_id': u'109399420', u'diversity': 0.3316818688890347}
{u'followers_count': 490507, u'_id': u'68577152', u'diversity': 0.2691131498470948}
{u'followers_count': 483260, u'_id': u'22175551', u'diversity': 0.354715637399532}
{u'followers_count': 438493, u'_id': u'1321629762', u'diversity': 0.43502210625498294}
{u'followers_count': 413092, u'_id': u'353060311', u'diversity': 0.24195838735568315}
{u'followers_count': 354218, u'_id': u'19276719', u'divers

In [38]:
top30 = db.retweets.distinct('userid')


In [39]:
top_lex = []
top_followers = []
for user in top30:
    print user
    ld = db.lexdiv2.find_one( {'user_id' : user}, {'diversity' : 1, 'followers_count' : 1, '_id' : 0})
    print ld
    top_lex.append(ld['followers_count'])
    top_followers.append(ld['diversity'])

    

26270913
{u'followers_count': 1012321, u'diversity': 0.22243024963289282}
2238576858
{u'followers_count': 927141, u'diversity': 0.4324224519940916}
487393551
{u'followers_count': 167964, u'diversity': 0.3072452861741712}
8521542
{u'followers_count': 173619, u'diversity': 0.24335319498664476}
840957998
{u'followers_count': 104065, u'diversity': 0.1903751290581622}
31451582
{u'followers_count': 1749003, u'diversity': 0.23825352618812326}
313162621
{u'followers_count': 62599, u'diversity': 0.7929203539823009}
330446733
{u'followers_count': 3065367, u'diversity': 0.31103000811030007}
313005140
{u'followers_count': 83110, u'diversity': 0.39528583039826604}
68577152
{u'followers_count': 490507, u'diversity': 0.2691131498470948}
1049422088
{u'followers_count': 33434, u'diversity': 0.34645340751043113}
353060311
{u'followers_count': 413092, u'diversity': 0.24195838735568315}
805230144
{u'followers_count': 2049528, u'diversity': 0.1545284664258129}
831981271
{u'followers_count': 237598, u'diver

In [40]:
top_lex

[1012321,
 927141,
 167964,
 173619,
 104065,
 1749003,
 62599,
 3065367,
 83110,
 490507,
 33434,
 413092,
 2049528,
 237598,
 3239,
 438493]